In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm


In [2]:
# Load the dataset
df = pd.read_csv("acute_oral_ld50_standard.csv")

# Check the shape and columns
print(df.shape)
print(df.columns)
df.head()


(13512, 12)
Index(['Unnamed: 0', 'Record_ID', 'Chemical_Name', 'CASRN', 'SMILES',
       'Species', 'Sex', 'Endpoint', 'Relation', 'Standard_Value',
       'Standard_Units', 'Reference'],
      dtype='object')


,Unnamed: 0,Record_ID,Chemical_Name,CASRN,SMILES,Species,Sex,Endpoint,Relation,Standard_Value,Standard_Units,Reference
0,0,acute_oral_12322,Captan-related-substance,NaN,NaN,Rat,Female,LD50,>,5000.0,mg/kg,Studies submitted to EPA in support of pestici...
1,1,acute_oral_12133,Tepraloxydim-5-hydroxy,15479-55-0,NaN,Rat,Female,LD50,>,5000.0,mg/kg,Studies submitted to EPA in support of pestici...
2,2,acute_oral_12132,Tepraloxydim-5-hydroxy,15479-55-0,NaN,Rat,Male/Female,LD50,>,5000.0,mg/kg,Studies submitted to EPA in support of pestici...
3,3,acute_oral_12894,Dichlorvos-related-substance,NaN,NaN,Rat,Male,LD50,=,3710.0,mg/kg,Studies submitted to EPA in support of pestici...
4,4,acute_oral_12323,Captan-related-substance,NaN,NaN,Rat,Male,LD50,>,5000.0,mg/kg,Studies submitted to EPA in support of pestici...


In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("acute_oral_ld50_standard.csv")

# Drop unneeded columns
df = df[['SMILES', 'Standard_Value', 'Endpoint', 'Species', 'Sex']].dropna(subset=['SMILES', 'Standard_Value'])

# Rename SMILES for consistency
df.rename(columns={'SMILES': 'smiles', 'Standard_Value': 'standard_value'}, inplace=True)

# Create binary toxicity label (1 = toxic, 0 = non-toxic)
# Threshold: LD50 ≤ 1000 mg/kg → toxic
df['label'] = np.where(df['standard_value'] <= 1000, 1, 0)

# Drop other metadata columns if you want only SMILES + label for fingerprint modeling
df_tox = df[['smiles', 'label']]

print("✅ Columns retained:", df.columns.tolist())
print("📊 Label distribution:\n", df_tox['label'].value_counts())
df_tox.head()


✅ Columns retained: ['smiles', 'standard_value', 'Endpoint', 'Species', 'Sex', 'label']
📊 Label distribution:
 label
0    7724
1    5464
Name: count, dtype: int64


,smiles,label
10,OC(C1=CC=CC=C1)C1=C(Cl)N=CC=C1,0
11,COC1=CC=C(C=C1)C1SCCC(O)=N1,0
12,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,0
13,CCC(=C)CCCC(C)COC(C)=O,0
14,CCCP(=O)(OC)SC,1


In [4]:
df = df[df['standard_value'] > 0]  # remove invalid zero values
df['label'] = np.where(df['standard_value'] <= 1000, 1, 0)

# Keep only smiles + label
df = df[['smiles', 'label']]
df.head()


,smiles,label
10,OC(C1=CC=CC=C1)C1=C(Cl)N=CC=C1,0
11,COC1=CC=C(C=C1)C1SCCC(O)=N1,0
12,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,0
13,CCC(=C)CCCC(C)COC(C)=O,0
14,CCCP(=O)(OC)SC,1


In [5]:
tqdm.pandas()

def smiles_to_morgan(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

df["fingerprint"] = df["smiles"].progress_apply(smiles_to_morgan)
df = df.dropna(subset=["fingerprint"])


[12:58:28] DEPRECATION WARNING: please use MorganGenerator                                   | 0/13188 [00:00<?, ?it/s]
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGenerator
[12:58:28] DEPRECATION WARNING: please use MorganGener

In [6]:
X = np.array([np.array(fp) for fp in df["fingerprint"]])
y = df["label"].values

print("Feature matrix shape:", X.shape)
print("Label distribution:\n", pd.Series(y).value_counts())


Feature matrix shape: (13177, 2048)
Label distribution:
 0    7713
1    5464
Name: count, dtype: int64


In [7]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print("After SMOTE:", X_res.shape, pd.Series(y_res).value_counts())


After SMOTE: (15426, 2048) 0    7713
1    7713
Name: count, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)



In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Random Forest Accuracy: 0.7521062864549579
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      1543
           1       0.73      0.80      0.76      1543

    accuracy                           0.75      3086
   macro avg       0.75      0.75      0.75      3086
weighted avg       0.75      0.75      0.75      3086



In [10]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.7987686325340246
              precision    recall  f1-score   support

           0       0.81      0.78      0.80      1543
           1       0.79      0.81      0.80      1543

    accuracy                           0.80      3086
   macro avg       0.80      0.80      0.80      3086
weighted avg       0.80      0.80      0.80      3086



In [11]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgb))
print(classification_report(y_test, y_pred_lgb))


[LightGBM] [Info] Number of positive: 6170, number of negative: 6170
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028491 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3828
[LightGBM] [Info] Number of data points in the train set: 12340, number of used features: 1914
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM Accuracy: 0.7916396629941672
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1543
           1       0.78      0.81      0.80      1543

    accuracy                           0.79      3086
   macro avg       0.79      0.79      0.79      3086
weighted avg       0.79      0.79      0.79      3086



C:\Users\anshul\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [12]:
df.to_csv("acute_oral_ld50_cleaned_ml_ready.csv", index=False)
print("✅ Saved as 'acute_oral_ld50_cleaned_ml_ready.csv'")


✅ Saved as 'acute_oral_ld50_cleaned_ml_ready.csv'


In [12]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, MACCSkeys, rdMolDescriptors

# ✅ Toxic alert SMARTS rules
TOX_ALERTS = {
    "nitro_aromatic": "[$([NX3](=O)=O)][cH]",
    "halogen_aromatic": "c[F,Cl,Br,I]",
    "epoxide": "C1OC1",
    "michael_acceptor": "[$([C]=[C]C=O)]",
    "pah": "a1aaaaa1",
    "primary_amine": "[NH2][CX4]",
    "secondary_amine": "[NH][CX4]",
}

alert_smarts = {k: Chem.MolFromSmarts(v) for k, v in TOX_ALERTS.items()}

def extract_features(smiles):
    m = Chem.MolFromSmiles(smiles)
    if m is None:
        return None
    
    # ✅ Standard properties
    mw = Descriptors.ExactMolWt(m)
    logp = Descriptors.MolLogP(m)
    tpsa = Chem.rdMolDescriptors.CalcTPSA(m)
    hba = rdMolDescriptors.CalcNumHBA(m)
    hbd = rdMolDescriptors.CalcNumHBD(m)
    rotb = rdMolDescriptors.CalcNumRotatableBonds(m)
    arom_rings = rdMolDescriptors.CalcNumAromaticRings(m)
    
    total_rings = rdMolDescriptors.CalcNumRings(m)
    max_ring_size = max((len(r) for r in m.GetRingInfo().AtomRings()), default=0)
    
    hetero = sum(1 for atom in m.GetAtoms() if atom.GetAtomicNum() not in [1,6])
    formal_charge = Chem.GetFormalCharge(m)
    frac_aromatic = arom_rings / total_rings if total_rings > 0 else 0
    
    props = np.array([
        mw, logp, tpsa, hba, hbd, rotb,
        arom_rings, frac_aromatic, max_ring_size,
        hetero, formal_charge
    ], dtype=np.float32)
    
    # ✅ Fingerprints
    fp4 = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
    fp6 = AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=2048)
    
    fp4 = np.frombuffer(fp4.ToBitString().encode("ascii"), dtype="S1").astype(np.uint8)-48
    fp6 = np.frombuffer(fp6.ToBitString().encode("ascii"), dtype="S1").astype(np.uint8)-48
    
    # ✅ MACCS keys
    maccs = MACCSkeys.GenMACCSKeys(m)
    maccs = np.array(list(maccs.ToBitString()), dtype=np.uint8)
    
    # ✅ Toxic alerts
    tox = []
    for patt in alert_smarts.values():
        tox.append(float(m.HasSubstructMatch(patt)))
    tox = np.array(tox, dtype=np.float32)
    
    return np.concatenate([fp4, fp6, maccs, props, tox])


def build_feature_matrix(df):
    X, keep_idx = [], []
    for i, smi in enumerate(df["smiles"]):
        f = extract_features(smi)
        if f is not None:
            X.append(f)
            keep_idx.append(i)
    return np.vstack(X).astype(np.float32), df.iloc[keep_idx].reset_index(drop=True)

# ✅ Load Dataset
df = pd.read_csv("acute_oral_ld50_standard.csv").dropna(subset=["smiles"])

X, df_clean = build_feature_matrix(df)

print("✅ Features shape:", X.shape)
print("✅ Cleaned dataset rows:", len(df_clean))


[17:24:03] Explicit valence for atom # 0 O, 3, is greater than permitted
[17:24:03] Explicit valence for atom # 0 O, 3, is greater than permitted


✅ Features shape: (13175, 4281)
✅ Cleaned dataset rows: 13175


In [13]:
from rdkit import Chem, RDLogger
RDLogger.DisableLog("rdApp.*")

def safe_mol_from_smiles(s):
    if not isinstance(s, str): return None
    try:
        m = Chem.MolFromSmiles(s, sanitize=True)
        return m
    except Exception:
        return None

# keep only valid molecules
df["rdmol"] = df["smiles"].map(safe_mol_from_smiles)
bad = df["rdmol"].isna().sum()
df = df.dropna(subset=["rdmol"]).reset_index(drop=True)
print(f"✅ valid mols: {len(df)} | ❌ dropped invalid: {bad}")


✅ valid mols: 13175 | ❌ dropped invalid: 2


In [16]:
import re
import numpy as np
import pandas as pd
from rdkit import Chem
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
from xgboost import XGBRegressor

# Assumes X (features) and df_clean (rows aligned to X) already exist.
# 1) Pick target column automatically (LD50-style)
def pick_target_column(df: pd.DataFrame) -> str:
    cand_patterns = [
        r"^ld[_\s\-]*50", r"ld50", r"oral[_\s\-]*ld50", r"standard[_\s\-]*value",
        r"tox[_\s\-]*value", r"value$"
    ]
    num_cols = [c for c in df.columns if np.issubdtype(df[c].dropna().dtype, np.number)]
    # rank by regex score, fallback to longest numeric column
    scores = []
    for c in num_cols:
        name = c.lower()
        score = max((re.search(p, name) is not None) for p in cand_patterns)
        scores.append((score, c))
    scores.sort(key=lambda t: (t[0], len(df[t[1]].dropna())), reverse=True)
    if not scores:
        raise KeyError("No numeric column found for target.")
    target_col = scores[0][1]
    return target_col

target_col = pick_target_column(df_clean)
print("✅ Using target column:", target_col)

# 2) Build chemotype groups (INCHIKEY14)
if "INCHIKEY14" in df_clean.columns:
    groups = df_clean["INCHIKEY14"].astype(str).values
else:
    def ikey14_from_smiles(s):
        try:
            m = Chem.MolFromSmiles(s)
            return Chem.inchi.MolToInchiKey(m)[:14] if m is not None else "UNK"
        except Exception:
            return "UNK"
    base_smiles_col = "smiles" if "smiles" in df_clean.columns else [c for c in df_clean.columns if "smile" in c.lower()][0]
    groups = df_clean[base_smiles_col].astype(str).apply(ikey14_from_smiles).values

# 3) Align X and y
y_all = pd.to_numeric(df_clean[target_col], errors="coerce").values
mask = ~np.isnan(y_all)
X_use = X[mask]
y_use = y_all[mask]
groups_use = np.asarray(groups)[mask]

print("Shapes -> X:", X_use.shape, "| y:", y_use.shape, "| groups:", groups_use.shape)

# 4) Group (chemotype) split
gss = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state=42)
tr_idx, te_idx = next(gss.split(X_use, groups=groups_use))
X_train, X_test = X_use[tr_idx], X_use[te_idx]
y_train_raw, y_test_raw = y_use[tr_idx], y_use[te_idx]

# 5) Log-transform LD50 (stabilize heavy tails)
y_train = np.log1p(y_train_raw)
y_test  = np.log1p(y_test_raw)

# 6) Train robust baseline
model = XGBRegressor(
    n_estimators=1200,
    learning_rate=0.045,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method="hist",
    random_state=42,
    n_jobs=-1,
)
model.fit(X_train, y_train)

# 7) Evaluate (back-transform)
pred_log = model.predict(X_test)
pred = np.expm1(pred_log)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr

# ensure 1D numpy arrays
y_test_raw = np.asarray(y_test_raw).ravel()
pred       = np.asarray(pred).ravel()

# RMSE without `squared=`
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr

# ensure 1D numpy arrays
y_test_raw = np.asarray(y_test_raw).ravel()
pred       = np.asarray(pred).ravel()

# RMSE without `squared=`
mse  = mean_squared_error(y_test_raw, pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test_raw, pred)
r2   = r2_score(y_test_raw, pred)

rho, _ = spearmanr(y_test_raw, pred)  # Spearman rank corr

print("\n📊 LD50 Regression (chemotype split)")
print(f"RMSE     : {rmse:.4f}")
print(f"MAE      : {mae:.4f}")
print(f"R^2      : {r2:.4f}")
print(f"Spearman : {rho:.4f}")


✅ Using target column: ld50_value
Shapes -> X: (13175, 4281) | y: (13175,) | groups: (13175,)

📊 LD50 Regression (chemotype split)
RMSE     : 4305.7499
MAE      : 1895.0818
R^2      : 0.1438
Spearman : 0.6595


In [17]:
# Log-scale evaluation
pred_log = np.log1p(pred)
true_log = np.log1p(y_test_raw)

rmse_log = np.sqrt(mean_squared_error(true_log, pred_log))
mae_log  = mean_absolute_error(true_log, pred_log)

print("RMSE_log:", rmse_log)
print("MAE_log:", mae_log)


RMSE_log: 1.3238993797206438
MAE_log: 0.9659083379460564


In [18]:
# Toxicity category mapping based on OECD acute oral hazard
def ld50_to_class(ld50):
    if ld50 <= 50:
        return "Very Toxic"
    elif ld50 <= 300:
        return "Toxic"
    elif ld50 <= 2000:
        return "Harmful"
    return "Low Toxicity"

df_results = df_clean.iloc[te_idx].copy()
df_results = df_results.reset_index(drop=True)

df_results["pred_ld50"] = pred.astype(float)
df_results["true_class"] = df_results[target_col].apply(ld50_to_class)
df_results["pred_class"] = df_results["pred_ld50"].apply(ld50_to_class)

# Confidence = how strongly LD50 deviates from boundary thresholds
def toxicity_confidence(ld50):
    if ld50 <= 50:
        return 1 - (ld50 / 50)
    elif ld50 <= 300:
        return 1 - (abs(ld50 - 175) / 125)
    elif ld50 <= 2000:
        return 1 - (abs(ld50 - 1150) / 850)
    return min((ld50 - 2000) / 2000, 1)

df_results["confidence"] = df_results["pred_ld50"].apply(toxicity_confidence).clip(lower=0)

# Export
df_results.to_excel("oral_ld50_predictions.xlsx", index=False)
print("✅ Saved: oral_ld50_predictions.xlsx")

print("\nSample Predictions:")
print(df_results[[
    "smiles", target_col, "pred_ld50", 
    "true_class", "pred_class",
    "confidence"
]].head())


✅ Saved: oral_ld50_predictions.xlsx

Sample Predictions:
                                              smiles  ld50_value    pred_ld50  \
0         CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC      2000.0  4318.395020   
1                                     CCCP(=O)(OC)SC        25.0    47.118469   
2           C*.C*.C1CCOC1 |lp:7:2,m:1:8.4.5.6,3:5.6|      4000.0  1261.913574   
3           ClC1=CC=C(C=C1)[C@@]12C[C@@H]1C(=O)NC2=O      1550.0  1354.364014   
4  [H][C@@]12C[C@@H](OC(C)=O)[C@@H](CN1CCC1=C2C=C...      1050.0  1460.814331   

     true_class    pred_class  confidence  
0       Harmful  Low Toxicity    1.000000  
1    Very Toxic    Very Toxic    0.057631  
2  Low Toxicity       Harmful    0.868337  
3       Harmful       Harmful    0.759572  
4       Harmful       Harmful    0.634336  


In [19]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(
    df_results["true_class"], 
    df_results["pred_class"], 
    labels=["Very Toxic","Toxic","Harmful","Low Toxicity"]
)

print("\n📊 Confusion Matrix:")
print(cm)

print("\n📊 Classification Report:")
print(classification_report(
    df_results["true_class"],
    df_results["pred_class"],
    digits=3
))



📊 Confusion Matrix:
[[ 71  79  54   8]
 [ 16 127 245  16]
 [  4  89 710 219]
 [  0  19 423 576]]

📊 Classification Report:
              precision    recall  f1-score   support

     Harmful      0.496     0.695     0.579      1022
Low Toxicity      0.703     0.566     0.627      1018
       Toxic      0.404     0.314     0.354       404
  Very Toxic      0.780     0.335     0.469       212

    accuracy                          0.559      2656
   macro avg      0.596     0.477     0.507      2656
weighted avg      0.584     0.559     0.554      2656



In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd

# Toxic vs Non-toxic threshold (OECD)
def tox_bin(ld50):
    return 1 if ld50 <= 300 else 0

true_bin = np.array([tox_bin(v) for v in y_test_raw])
pred_bin = np.array([tox_bin(v) for v in pred])

accuracy  = accuracy_score(true_bin, pred_bin)
precision = precision_score(true_bin, pred_bin, zero_division=0)
recall    = recall_score(true_bin, pred_bin, zero_division=0)
f1        = f1_score(true_bin, pred_bin, zero_division=0)
auc = roc_auc_score(true_bin, -pred)  # invert LD50 scale

cm = confusion_matrix(true_bin, pred_bin)

print("\n📊 Acute Oral Toxicity (Binary Classification)")
print(f"Accuracy  : {accuracy:.3f}")
print(f"Precision : {precision:.3f}")
print(f"Recall    : {recall:.3f}")
print(f"F1 Score  : {f1:.3f}")
print(f"ROC-AUC   : {auc:.3f}")
print("\nConfusion Matrix (rows: True, cols: Pred):")
print(cm)



📊 Acute Oral Toxicity (Binary Classification)
Accuracy  : 0.836
Precision : 0.723
Recall    : 0.476
F1 Score  : 0.574
ROC-AUC   : 0.844

Confusion Matrix (rows: True, cols: Pred):
[[1928  112]
 [ 323  293]]
